## Imports

In [3]:
import pandas as pd
import os
import numpy as np
from pyvis.network import Network
from neo4j import GraphDatabase


## Functions 

### Find Each Table's primary key

In [4]:
def pk_finder(name, df):
    for i in range(len(df.columns)):
        col = df.iloc[:,i]
        #if(col.size == col.drop_duplicates().size):
        if(col.nunique() == col.size):
            pk[name] = df.columns[i] 
            break
        

### Find Each Table's foreign key


In [5]:
def fk_finder(name, df):
    for i in range(len(df.columns)):
        col_name = df.columns[i]
        col = df.iloc[:,i]
        if(list(df.columns)[i] not in pk):
            flag = True
            for j in pk: #loops on the key j = table names
                if j != name:
                    primary_key_column = All_dfs[j][(pk[j])]
                    if primary_key_column.dtype == col.dtype:
                        status = col.isin(primary_key_column)
                        if status[status== False].size == 0:
                            fk[name][col_name] = j 
                            ref_in[j].add(name)
                            break

### Assign Tables to nodes or edges

In [6]:
def nodes_edges(fk):
    for f in fk:
        if len(fk[f]) == 2:
            inner_dict = list(fk[f])
            edges[f] = fk[f]
        elif len(fk[f]) == 1:
            properties[f] = fk[f]
        else:
            nodes[f] = All_dfs[f]
#         else:
#             if len(fk[f]) == 1:
#                 properties[f] = fk[f]
            
#             nodes[f] = All_dfs[f]
    # for f in fk:
    #     if((f not in edges) ):
    #         nodes[f] = All_dfs[f]
    #         #nodes.append(f)
        

### Create Nodes Table

In [7]:
def add_nodes(nodes, nodesTable):
    for n in range(len(nodes)):
        for index, row in nodes[list(nodes.keys())[n]].iterrows():
            column_names = list(nodes[list(nodes.keys())[n]].columns) #get column names
            att = {}
            for i in range(1, len(column_names)):
                att[column_names[i]] = nodes[list(nodes.keys())[n]].iloc[index, i]
            newRow = [{'Label': list(nodes.keys())[n] , 'ID': nodes[list(nodes.keys())[n]].iloc[index,0] , 'Attributes': att}]
            tmp = pd.DataFrame(newRow)
            nodesTable = pd.concat([nodesTable, tmp], ignore_index=True)
    return nodesTable


### Create Edges Table

In [8]:
def add_edges(edges,edgesTable,pk,fk,ref_in):
    for e in edges:
        df = All_dfs[e] #df of the cur edge
        for r in range(len(df)):
            label = list(edges[e].keys())
            from_id = nodesTable[(nodesTable["Label"] == edges[e][label[0]]) & (df.loc[r,label[0]] == nodesTable["ID"])]
            from_id = from_id.index[0]
            to_id = nodesTable[(nodesTable["Label"] == edges[e][label[1]]) & (df.loc[r,label[1]] == nodesTable["ID"])]
            to_id = to_id.index[0]
            pk_col = pk[e]
#             primary_key = df[pk_col].iloc[r]
            primary_key = df.loc[r,pk_col]
            att = convert_prop(ref_in,e,primary_key,fk)
            newRow = [{'From_Node_ID': from_id, 'To_Node_ID': to_id, 'order/service' : att}]
            tmp = pd.DataFrame(newRow)
            edgesTable = pd.concat([edgesTable, tmp], ignore_index=True)
    return edgesTable

### Properties of the Edge

In [9]:
def convert_prop(ref_in,edge_key, pk_value,fk):
    for referenced_table_name in ref_in[edge_key]:
        referenced_table = All_dfs[referenced_table_name]
        fk_of_ref_table = fk[referenced_table_name]
        for foreign_key in fk_of_ref_table:
            if fk_of_ref_table[foreign_key] == edge_key:
#                 referenced_table[referenced_table[foreign_key]== id]
                all_occurances_df = referenced_table[referenced_table[foreign_key]== pk_value].drop([foreign_key],axis=1)
                return all_occurances_df.to_dict('records')
                

### generate list of n colors

In [10]:
import random
get_colors = lambda n: list(map(lambda i: "#" + "%06x" % random.randint(0, 0xFFFFFF),range(n)))
# get_colors(5) # sample return:  ['#8af5da', '#fbc08c', '#b741d0', '#e599f1', '#bbcb59', '#a2a6c0']

### Draw Pyvis graph

In [11]:
def draw_graph_pyvis(nodes_table,edges_table):

    nodes_name = [(x[1]+'_'+str(x[0])).capitalize() for x in zip(nodes_table.ID,nodes_table.Label)]
    indices = list(map(lambda x: int(x),list(nodes_table.index)))
    nodes_attributes = list(map(lambda x: str(x).replace(',',"\n"),list(nodes_table.Attributes)))
    nodes_tables = list(nodes_df.Label.unique())
    colors = get_colors(len(nodes_name))
    nodes_color = list(map(lambda x: colors[nodes_tables.index(x)],list(nodes_table.Label)))
    
    g = Network(width='100%')
#     g.barnes_hut()
    g.add_nodes(indices,
                title= nodes_attributes,
                label=nodes_name,
                color=nodes_color,
                )
    for e in range(len(edges_table)):
        src = int(edges_table.iloc[e]["From"])
        dst = int(edges_table.iloc[e]["To"])
        g.add_edge(src,dst,weight=5)

    print("Number of Nodes: ",len(g.get_nodes()))
    print("Number of Edges: ",len(g.get_edges()))
    

    g.show('test.html')




### Draw Graph Neo4j

#### prepare satements

In [12]:
def draw_graph_Neo4j(nodes_table,edges_table):

    transaction_execution_commands = []
    # creating a statement for each node (row in node_table)
    for i, node in nodes_table.iterrows():
        label = node["Label"]
        index = i
        ID = node["ID"]
        attributes = node["Attributes"]
        neo4j_create_statemenet = Create_Statement(label,i,ID,attributes)
#         print(neo4j_create_statemenet)
        transaction_execution_commands.append(neo4j_create_statemenet)

     
    execute_transactions(transaction_execution_commands)
 

#### Connect and execute statements

In [13]:
def execute_transactions(transaction_execution_commands):
    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "password"))
    session = data_base_connection.session()    
    for i in transaction_execution_commands:
        session.run(i)

 

#### Create a "Create statement for nodes"

In [14]:
def Create_Statement(label,index,ID,attributes):
    return f"CREATE (x:{label} {'{'}name: {get_name(label,ID)},index:{index},ID:{ID} ,{destrucure_dict(attributes)} {'}'})"


In [15]:
def get_name(label,ID):
    return '\"' + label.capitalize()+'_'+str(ID)+'\"'

#### prepare attributes for create statement

In [16]:
def destrucure_dict(dic):
    s = ""
    for key in dic:
        value = dic[key]
        if isinstance (value,str):
            value = "\"" + value +"\""
        s=s + str(key) + ":" + str(value)+","
    return s[:-1]

## Global Variables

In [17]:
# {Table name: df}
All_dfs = {}
# {Table name: Table's Primary key}
pk = {}
# {Table name: {Column Name: Referenced Table}}
fk = {}
# {Table name: set(Tables that references the table)}
ref_in = {}
# {Table name: df}
nodes = {}
# {Table name: {Column Name: Referenced Table}}
edges = {}
# _
properties = {}
# Nodes_df
nodesTable = pd.DataFrame(columns=['Label', 'ID', 'Attributes'])
# Edges_df
edgesTable = pd.DataFrame(columns = ['From_Node_ID', 'To_Node_ID', 'order/service'])
# Properties_df
propertiesTable = pd.DataFrame(columns = [['index','From', 'Label', 'Attributes']])
#Label = ['facilities'/ 'order' / 'service']


# Nodes
# Edges ['From_Node_ID', 'To_Node_ID']
# Properties ['Label', 'ID', 'Attributes']
# (^Nodes in graph)
# Relationship ["Relation_name","from","to"]


nodes_df = pd.DataFrame(columns=['Label', 'ID', 'Attributes'])
edges_df = pd.DataFrame(columns=['From','To'])


## Reading Data Set

 creating a dictionary where
 key: first word of the table name 
 value: df 

In [18]:
import os
path_of_the_directory = './DataSet/'
ext = ('.csv')
for file in os.listdir(path_of_the_directory):
    if file.endswith(ext):
        print(file) 
        temp = (file.replace("_"," ").replace("."," ").split(" ")[0].lower())
        All_dfs[temp] = pd.read_csv(path_of_the_directory+file)
    else:
        continue

customer_data.csv
ExternalOrders_data.csv
ExternalServices_data.csv
ExternalShipments_data.csv
ExternalTransactions.csv
facilities_data.csv
InternalOrders_data.csv
InternalServices_data.csv
InternalShipments_data.csv
InternalTransactions.csv
Manufacturing_data.csv
Products_data.csv
Retailer_data.csv
Supplier_data.csv
warehouses_data.csv


### Splitting Shipments tables

In [19]:
SRIntShip = All_dfs["internalshipments"].query('from_to_where == "SR"')
SRIntShip = SRIntShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SRIntShip
RCExtShip = All_dfs["externalshipments"].query('from_to_where == "RC"')
RCExtShip = RCExtShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#RCExtShip
SSIntShip = All_dfs["internalshipments"].query('from_to_where == "SS"')
SSIntShip = SSIntShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SSIntShip
SCExtShip = All_dfs["externalshipments"].query('from_to_where == "SC"')
SCExtShip = SCExtShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SCExtShip
All_dfs["RCExtShip".lower()] = RCExtShip
All_dfs["SCExtShip".lower()] = SCExtShip
All_dfs["SRIntShip".lower()] = SRIntShip
All_dfs["SSIntShip".lower()] = SSIntShip

### Splitting Orders Tables

In [20]:
filter_list = All_dfs["rcextship"]["ExtShip_id"]
All_dfs["rcextorders"] = All_dfs["externalorders"][All_dfs["externalorders"].ExtShip_id.isin(filter_list)].reset_index(drop=True)
filter_list = All_dfs["scextship"]["ExtShip_id"]
All_dfs["scextorders"] = All_dfs["externalorders"][All_dfs["externalorders"].ExtShip_id.isin(filter_list)].reset_index(drop=True)
#All_dfs["scextorders"] = All_dfs["externalorders"].query("ExtShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["srintship"]["IntShip_id"]
All_dfs["srintorders"] = All_dfs["internalorders"][All_dfs["internalorders"].IntShip_id.isin(filter_list)].reset_index(drop=True)
#All_dfs["srintorders"] = All_dfs["internalorders"].query("IntShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["ssintship"]["IntShip_id"]
All_dfs["ssintorders"] = All_dfs["internalorders"][All_dfs["internalorders"].IntShip_id.isin(filter_list)].reset_index(drop=True)
#All_dfs["ssintorders"] = All_dfs["internalorders"].query("IntShip_id.isin(@filter_list)").reset_index(drop=True)

### Removing Redundant Tables

In [21]:
All_dfs.pop("internalshipments")
All_dfs.pop("externalshipments")
All_dfs.pop("externalorders")
All_dfs.pop("internalorders")
All_dfs.keys()

dict_keys(['customer', 'externalservices', 'externaltransactions', 'facilities', 'internalservices', 'internaltransactions', 'manufacturing', 'products', 'retailer', 'supplier', 'warehouses', 'rcextship', 'scextship', 'srintship', 'ssintship', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

### Execlude Tables

In [22]:
All_dfs = {key: value for key, value in All_dfs.items() 
           if key not in ["products","warehouses","manufacturing"]}
All_dfs.keys()

dict_keys(['customer', 'externalservices', 'externaltransactions', 'facilities', 'internalservices', 'internaltransactions', 'retailer', 'supplier', 'rcextship', 'scextship', 'srintship', 'ssintship', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

### Initialize fk and ref_in

In [23]:
table_name = list(All_dfs.keys())
for table_name in All_dfs:
    fk[table_name] = {}
    ref_in[table_name] = set()

### Get Primary key for each table

In [24]:
for t in All_dfs:
    pk_finder(t,All_dfs[t])
pk

{'customer': 'cust_id',
 'externalservices': 'ExtServ_id',
 'externaltransactions': 'ExtTran_id',
 'facilities': 'fac_id',
 'internalservices': 'IntServ_id',
 'internaltransactions': 'IntTran_id',
 'retailer': 'retailer_id',
 'supplier': 'supp_id',
 'rcextship': 'ExtShip_id',
 'scextship': 'ExtShip_id',
 'srintship': 'IntShip_id',
 'ssintship': 'IntShip_id',
 'rcextorders': 'ExtOrders_id',
 'scextorders': 'ExtOrders_id',
 'srintorders': 'IntOrders_id',
 'ssintorders': 'IntOrders_id'}

### Get Foreing key for each table

In [25]:
for t in All_dfs:
    fk_finder(t,All_dfs[t])
fk

{'customer': {},
 'externalservices': {'ExtTrans_id': 'externaltransactions'},
 'externaltransactions': {'CompFrom': 'supplier', 'Custto': 'customer'},
 'facilities': {'supplier_id': 'supplier'},
 'internalservices': {'IntTrans_id': 'internaltransactions'},
 'internaltransactions': {'CompFrom': 'supplier', 'Compto': 'supplier'},
 'retailer': {},
 'supplier': {},
 'rcextship': {'factoryIds/retailerIds': 'retailer', 'idsTo': 'customer'},
 'scextship': {'factoryIds/retailerIds': 'supplier', 'idsTo': 'customer'},
 'srintship': {'listSuppIds': 'supplier', 'factoryIds': 'retailer'},
 'ssintship': {'listSuppIds': 'supplier', 'factoryIds': 'supplier'},
 'rcextorders': {'ExtShip_id': 'rcextship'},
 'scextorders': {'ExtShip_id': 'scextship'},
 'srintorders': {'IntShip_id': 'srintship'},
 'ssintorders': {'IntShip_id': 'ssintship'}}

In [26]:
ref_in

{'customer': {'externaltransactions', 'rcextship', 'scextship'},
 'externalservices': set(),
 'externaltransactions': {'externalservices'},
 'facilities': set(),
 'internalservices': set(),
 'internaltransactions': {'internalservices'},
 'retailer': {'rcextship', 'srintship'},
 'supplier': {'externaltransactions',
  'facilities',
  'internaltransactions',
  'scextship',
  'srintship',
  'ssintship'},
 'rcextship': {'rcextorders'},
 'scextship': {'scextorders'},
 'srintship': {'srintorders'},
 'ssintship': {'ssintorders'},
 'rcextorders': set(),
 'scextorders': set(),
 'srintorders': set(),
 'ssintorders': set()}

### Determine nodes and edges

In [27]:
nodes_edges(fk)

In [28]:
nodes.keys()

dict_keys(['customer', 'retailer', 'supplier'])

In [29]:
edges

{'externaltransactions': {'CompFrom': 'supplier', 'Custto': 'customer'},
 'internaltransactions': {'CompFrom': 'supplier', 'Compto': 'supplier'},
 'rcextship': {'factoryIds/retailerIds': 'retailer', 'idsTo': 'customer'},
 'scextship': {'factoryIds/retailerIds': 'supplier', 'idsTo': 'customer'},
 'srintship': {'listSuppIds': 'supplier', 'factoryIds': 'retailer'},
 'ssintship': {'listSuppIds': 'supplier', 'factoryIds': 'supplier'}}

In [30]:
properties

{'externalservices': {'ExtTrans_id': 'externaltransactions'},
 'facilities': {'supplier_id': 'supplier'},
 'internalservices': {'IntTrans_id': 'internaltransactions'},
 'rcextorders': {'ExtShip_id': 'rcextship'},
 'scextorders': {'ExtShip_id': 'scextship'},
 'srintorders': {'IntShip_id': 'srintship'},
 'ssintorders': {'IntShip_id': 'ssintship'}}

### Create NodesTable

In [31]:
nodesTable = add_nodes(nodes,nodesTable)
nodesTable

,Label,ID,Attributes
0,customer,33736,"{'gender': 'F', 'first_name': 'Nadia', 'last_n..."
1,customer,42959,"{'gender': 'M', 'first_name': 'Eric', 'last_na..."
2,customer,50653,"{'gender': 'F', 'first_name': 'Alessia', 'last..."
3,customer,82485,"{'gender': 'M', 'first_name': 'Alexei', 'last_..."
4,customer,56609,"{'gender': 'F', 'first_name': 'Brenda', 'last_..."
...,...,...,...
595,supplier,90256,"{'supplier_name': 'Förster', 'country': 'Ethio..."
596,supplier,35266,"{'supplier_name': 'Gotthard Zahn GmbH', 'count..."
597,supplier,52647,"{'supplier_name': 'Cook-Webb', 'country': 'Mor..."
598,supplier,13355,"{'supplier_name': 'Shah-Lowe', 'country': 'Cen..."


### Create Edges Table

In [32]:
edgesTable = add_edges(edges, edgesTable,pk,fk,ref_in)
edgesTable

,From_Node_ID,To_Node_ID,order/service
0,484,140,[]
1,426,72,"[{'ExtServ_id': 8658, 'placed_when': '1998-06-..."
2,465,2,[]
3,374,137,[]
4,475,92,"[{'ExtServ_id': 7764, 'placed_when': '1973-08-..."
...,...,...,...
395,459,348,"[{'IntOrders_id': 2751, 'quantity': 734, 'plac..."
396,321,471,"[{'IntOrders_id': 1084, 'quantity': 784, 'plac..."
397,343,351,"[{'IntOrders_id': 4939, 'quantity': 819, 'plac..."
398,321,394,"[{'IntOrders_id': 6448, 'quantity': 712, 'plac..."


In [33]:
# nodes_df = pd.DataFrame(columns=['Label', 'ID', 'Attributes'])
# edges_df = pd.DataFrame(columns=['From','To'])
all_nodes = list(nodes.keys())
for n in range(len(nodes)):
      for index, row in nodes[all_nodes[n]].iterrows():
          column_names = list(nodes[all_nodes[n]].columns) #get column names
          att = {}
          for i in range(1, len(column_names)):
              att[column_names[i]] = nodes[all_nodes[n]].iloc[index, i]
          newRow = [{'Label': all_nodes[n] , 'ID': nodes[all_nodes[n]].iloc[index,0] , 'Attributes': att}]
          tmp = pd.DataFrame(newRow)
          nodes_df = pd.concat([nodes_df, tmp], ignore_index=True)

In [34]:
nodes_df.iloc[50:]

,Label,ID,Attributes
50,customer,11346,"{'gender': 'M', 'first_name': 'Mitchell', 'las..."
51,customer,78650,"{'gender': 'M', 'first_name': 'Mohamed', 'last..."
52,customer,87252,"{'gender': 'M', 'first_name': 'Robert', 'last_..."
53,customer,24338,"{'gender': 'M', 'first_name': 'Sante', 'last_n..."
54,customer,18213,"{'gender': 'M', 'first_name': 'Friedo', 'last_..."
...,...,...,...
595,supplier,90256,"{'supplier_name': 'Förster', 'country': 'Ethio..."
596,supplier,35266,"{'supplier_name': 'Gotthard Zahn GmbH', 'count..."
597,supplier,52647,"{'supplier_name': 'Cook-Webb', 'country': 'Mor..."
598,supplier,13355,"{'supplier_name': 'Shah-Lowe', 'country': 'Cen..."


In [35]:
all_edges = list(edges.keys())
for n in range(len(edges)):
      Edge_name = all_edges[n]
      
      foreign_keys = list(fk[Edge_name].keys())

      from_col = foreign_keys[0]
      from_table_name = fk[Edge_name][from_col]
      from_df = All_dfs[from_table_name]
      from_df_pk = pk[from_table_name]
      
      to_col = foreign_keys[-1]
      to_table_name = fk[Edge_name][to_col]
      to_df = All_dfs[to_table_name]
      to_df_pk = pk[to_table_name]
      
      column_names = list(All_dfs[Edge_name].columns) #get column names
      for index, _ in All_dfs[Edge_name].iterrows():
          att = {}
          from_ref_id,to_ref_id = None, None
          
          
          for i in range(1, len(column_names)):
            column_name = column_names[i]
            
            
            if column_name not in foreign_keys:
              att[column_name] = All_dfs[Edge_name].iloc[index, i]
          
            else:
              reference_id = All_dfs[Edge_name].iloc[index, i]
              if column_name == from_col:
                # from_ref_id = from_df[from_df[from_df_pk] == reference_id].index[0]
                from_ref_id = reference_id
              else:
                # to_ref_id = to_df[to_df[to_df_pk] == reference_id].index[0]
                to_ref_id = reference_id

          # Adding new entry to node tabel
          newRow = [{'Label': Edge_name , 'ID': All_dfs[Edge_name].iloc[index,0] , 'Attributes': att}]
          tmp = pd.DataFrame(newRow)
          nodes_df = pd.concat([nodes_df, tmp], ignore_index=True)
          edge_node_index = len(nodes_df)-1
          # print(nodes_df.iloc[len(nodes_df)-1],All_dfs[Edge_name].iloc[index,0] )

          # creating two edges, one from the from_node to the edge node and one from edge node to to_node
          from_node_id = nodes_df[(nodes_df['Label']==from_table_name) &  (nodes_df['ID']==from_ref_id)].index[0]
          to_node_id =  nodes_df[(nodes_df['Label']==to_table_name) &  (nodes_df['ID']==to_ref_id)].index[0] 
          
          # from ---> edge 
          new_from_edge_row = [{'From': from_node_id , 'To':edge_node_index }]
          tmp = pd.DataFrame(new_from_edge_row)
          edges_df = pd.concat([edges_df, tmp], ignore_index=True)
          # edge --->to
          new_to_edge_row = [{'From': edge_node_index , 'To':  to_node_id}]
          tmp = pd.DataFrame(new_to_edge_row)
          edges_df = pd.concat([edges_df, tmp], ignore_index=True)



In [36]:
edges_df

,From,To
0,484,600
1,600,140
2,426,601
3,601,72
4,465,602
...,...,...
795,997,351
796,321,998
797,998,394
798,306,999


In [37]:
all_properties = list(properties.keys())
for property_name in all_properties:
  property_df = All_dfs[property_name]
      
  foreign_keys = list(fk[property_name].keys())

  fk_col = foreign_keys[0]
  referenced_table_name = fk[property_name][fk_col]
  referenced_table_df = All_dfs[referenced_table_name]
  referenced_table_pk = pk[referenced_table_name]
  
  column_names = list(property_df.columns) #get column names
  
  for index, _ in property_df.iterrows():    
      att = {}
      reference_id = None;      
      
      for i in range(1, len(column_names)):
        column_name = column_names[i]
        
        
        if column_name not in foreign_keys:
          att[column_name] = property_df.iloc[index, i]
      
        else:
          # capturing foreign key value
          reference_id = property_df.iloc[index, i]

      # Adding new entry to node tabel
      newRow = [{'Label': property_name , 'ID': property_df.iloc[index,0] , 'Attributes': att}]
      tmp = pd.DataFrame(newRow)
      nodes_df = pd.concat([nodes_df, tmp], ignore_index=True)
      property_node_index = len(nodes_df)-1
      # print(nodes_df.iloc[len(nodes_df)-1],property_df.iloc[index,0] )

      # creating two edges, one from the from_node to the edge node and one from edge node to to_node
      referenced_node_id = nodes_df[(nodes_df['Label']==referenced_table_name) &  (nodes_df['ID']==reference_id)].index[0] 
      
      new_property_edge_row = [{'From': referenced_node_id , 'To':  property_node_index}]
      tmp = pd.DataFrame(new_property_edge_row)
      edges_df = pd.concat([edges_df, tmp], ignore_index=True)



In [38]:
edges_df

,From,To
0,484,600
1,600,140
2,426,601
3,601,72
4,465,602
...,...,...
3195,951,3395
3196,992,3396
3197,970,3397
3198,992,3398


In [45]:
draw_graph_pyvis(nodes_df,edges_df)

Number of Nodes:  3400
Number of Edges:  3200


In [40]:
sum_nodes = 0
sum_edges = 0
for key in All_dfs:
    table_size = len(All_dfs[key])
    print(key, table_size)
    sum_nodes += table_size
    if(key in all_edges):
        sum_edges += table_size*2
    if(key in all_properties ):
        sum_edges += table_size
print("Number Of Nodes: ",sum_nodes)
print("Number Of Edges: ",sum_edges)

customer 150
externalservices 100
externaltransactions 100
facilities 2000
internalservices 100
internaltransactions 100
retailer 150
supplier 300
rcextship 51
scextship 49
srintship 51
ssintship 49
rcextorders 52
scextorders 48
srintorders 54
ssintorders 46
Number Of Nodes:  3400
Number Of Edges:  3200


In [46]:
nodes_df

,Label,ID,Attributes
0,customer,33736,"{'gender': 'F', 'first_name': 'Nadia', 'last_n..."
1,customer,42959,"{'gender': 'M', 'first_name': 'Eric', 'last_na..."
2,customer,50653,"{'gender': 'F', 'first_name': 'Alessia', 'last..."
3,customer,82485,"{'gender': 'M', 'first_name': 'Alexei', 'last_..."
4,customer,56609,"{'gender': 'F', 'first_name': 'Brenda', 'last_..."
...,...,...,...
3395,ssintorders,7965,"{'quantity': 259, 'placed_when': '2015-07-26',..."
3396,ssintorders,3273,"{'quantity': 414, 'placed_when': '2003-03-01',..."
3397,ssintorders,5047,"{'quantity': 389, 'placed_when': '1973-11-01',..."
3398,ssintorders,5886,"{'quantity': 106, 'placed_when': '2005-06-09',..."


In [47]:
edges_df

,From,To
0,484,600
1,600,140
2,426,601
3,601,72
4,465,602
...,...,...
3195,951,3395
3196,992,3396
3197,970,3397
3198,992,3398


In [ ]:
draw_graph_pyvis(nodes_df,edges_df)

In [ ]:
import time
for property in range(len(list(properties.keys()))):
  table_name = "facilities"
  if(list(properties.keys())[property] == table_name):
    referencedTableName = properties[table_name][list(properties[table_name].keys())[0]]
    if(referencedTableName in nodes.keys()):
      print("Referenced Table is Node")
    elif(referencedTableName in edges.keys()):
      print("Referenced Table is Edge")
    referencedColName = list(properties[table_name].keys())[0]
    fac_df = All_dfs[list(properties.keys())[property]]
    time.sleep(1)
    for index in range(fac_df.shape[0]):
      fac = fac_df.iloc[index]
      fromId = fac[pk[list(properties.keys())[property]]]
      toId = fac[referencedColName]
      print(fromId)
      print(toId)
      print("---------")


In [ ]:
for property in range(len(list(properties.keys()))):
  table_name = 'facilities'
  if(list(properties.keys())[property] == table_name):
    referencedTableName = fk[table_name][list(properties[table_name].keys())[0]]
    if(referencedTableName in nodes.keys()):
      print("Referenced Table is Node")
    elif(referencedTableName in edges.keys()):
      print("Referenced Table is Edge")
    referencedColName = list(properties[table_name].keys())[0]
    fac_df = All_dfs[list(properties.keys())[property]]
    for index in range(fac_df.shape[0]):
      fac = fac_df.iloc[index]
      fromId = fac[pk[list(properties.keys())[property]]]
      toId = fac[referencedColName]
      print(fromId)
      print(toId)
      print("---------")